In [1]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility
import time
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda
from keras.layers import Conv1D,MaxPooling1D, Flatten

from keras.utils import multi_gpu_model
from keras import backend as K
from sklearn.cross_validation import train_test_split

import pandas as pd
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import Normalizer
from keras.utils import np_utils

import h5py

X_train = np.load("20170827_mawilab_flow_000features.pkl")
Y_train = np.load("20170827_mawilab_flow_000labels.pkl")

scaler = Normalizer().fit(X_train)
X_train = scaler.transform(X_train)
scaler = Normalizer().fit(Y_train)
Y_train = scaler.transform(Y_train)

# Test Set 1 : 20170827_mawilab_flow_001
X_test1 = np.load("20170827_mawilab_flow_001features.pkl")
Y_test1 = np.load("20170827_mawilab_flow_001labels.pkl")
scaler = Normalizer().fit(X_test1)
X_test1 = scaler.transform(X_test1)
scaler = Normalizer().fit(Y_test1)
Y_test1 = scaler.transform(Y_test1)

# Test Set 2 : 20170827_mawilab_flow_002
X_test2 = np.load("20170827_mawilab_flow_002features.pkl")
Y_test2 = np.load("20170827_mawilab_flow_002labels.pkl")
scaler = Normalizer().fit(X_test2)
X_test2 = scaler.transform(X_test2)
scaler = Normalizer().fit(Y_test2)
Y_test2 = scaler.transform(Y_test2)

# Test Set 3 : 20170827_mawilab_flow_003
X_test3 = np.load("20170827_mawilab_flow_003features.pkl")
Y_test3 = np.load("20170827_mawilab_flow_003labels.pkl")
scaler = Normalizer().fit(X_test3)
X_test3 = scaler.transform(X_test3)
scaler = Normalizer().fit(Y_test3)
Y_test3 = scaler.transform(Y_test3)

# Test Set 4: 20170827_mawilab_flow_004
X_test4 = np.load("20170827_mawilab_flow_004features.pkl")
Y_test4 = np.load("20170827_mawilab_flow_004labels.pkl")
scaler = Normalizer().fit(X_test4)
X_test4 = scaler.transform(X_test4)
scaler = Normalizer().fit(Y_test4)
Y_test4 = scaler.transform(Y_test4)

# reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1],1))
X_test1 = np.reshape(X_test1, (X_test1.shape[0],X_test1.shape[1],1))
X_test2 = np.reshape(X_test2, (X_test2.shape[0],X_test2.shape[1],1))
X_test3 = np.reshape(X_test3, (X_test3.shape[0],X_test3.shape[1],1))
X_test4 = np.reshape(X_test4, (X_test4.shape[0],X_test4.shape[1],1))


Using TensorFlow backend.
C:\Anaconda\envs\tensorflow\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype uint8 was converted to float64 by the normalize function.
  warnings.warn(msg, _DataConversionWarning)


In [2]:
cnn1d_1 = Sequential([Conv1D(64, 3, padding="same",input_shape=(16, 1)),
    Activation('relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(2),
    Activation('sigmoid'),
])
print(cnn1d_1.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 16, 64)            256       
_________________________________________________________________
activation_1 (Activation)    (None, 16, 64)            0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 8, 64)             0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               65664     
_________________________________________________________________
activation_2 (Activation)    (None, 128)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
__________

In [4]:
cnn1d_1 = multi_gpu_model(cnn1d_1, gpus=8)
cnn1d_1.compile(loss="binary_crossentropy", optimizer="adam",metrics=['accuracy'])
# train
start_time = time.time()
cnn1d_1.fit(X_train, Y_train, batch_size=64, validation_data=(X_test2, Y_test2) ,epochs=10)
end_time = time.time() 
print("Total time taken to train the Training model is", (end_time - start_time))
# serialize model to JSON
model_json = cnn1d_1.to_json()
with open("20170827_mawilab_flow_000.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
cnn1d_1.save_weights("20170827_mawilab_flow_000.h5")
print("Saved model to disk")

Train on 38361 samples, validate on 47265 samples
Epoch 1/10
38361/38361 [==============================] - 3s 88us/step - loss: 0.4380 - acc: 0.7452 - val_loss: 0.4691 - val_acc: 0.7448
Epoch 2/10
38361/38361 [==============================] - 3s 77us/step - loss: 0.4361 - acc: 0.7454 - val_loss: 0.4702 - val_acc: 0.7441
Epoch 3/10
38361/38361 [==============================] - 3s 77us/step - loss: 0.4343 - acc: 0.7457 - val_loss: 0.4608 - val_acc: 0.7457
Epoch 4/10
38361/38361 [==============================] - 3s 81us/step - loss: 0.4328 - acc: 0.7464 - val_loss: 0.4655 - val_acc: 0.7456
Epoch 5/10
38361/38361 [==============================] - 3s 83us/step - loss: 0.4308 - acc: 0.7470 - val_loss: 0.4592 - val_acc: 0.7456
Epoch 6/10
38361/38361 [==============================] - 3s 80us/step - loss: 0.4304 - acc: 0.7476 - val_loss: 0.4525 - val_acc: 0.7465
Epoch 7/10
38361/38361 [==============================] - 3s 83us/step - loss: 0.4302 - acc: 0.7476 - val_loss: 0.4537 - val_acc